In [1]:
%matplotlib inline

In [2]:
import glob
import gzip
import json

import numpy as np
import pandas as pd

In [13]:
run_title = "baseline"

In [14]:
category_df = pd.read_csv("trec2019_test_results_run_%s.csv" % run_title, dtype={
    "tweet_id": np.int64,
})
priority_df = pd.read_csv("trec2019_test_results_priority_run_%s.csv" % run_title, dtype={
    "tweet_id": np.int64,
})

In [15]:
topic_num_to_id_map = {}

with open("TRECIS-2019-A-Test.topics", "r") as in_file:
    topic_num = ""
    topic_id = ""
    
    for line in in_file:
        
        if line.strip() == "</top>":
            topic_num_to_id_map[topic_id] = topic_num
        
        if line.startswith("<num>"):
            topic_num = line.partition(">")[-1].partition("<")[0]
              
        if line.startswith("<dataset>"):
            topic_id = line.partition(">")[-1].partition("<")[0]

In [16]:
topic_num_to_id_map

{'floodChoco2019': 'TRECIS-CTIT-H-Test-022',
 'fireAndover2019': 'TRECIS-CTIT-H-Test-023',
 'earthquakeCalifornia2014': 'TRECIS-CTIT-H-Test-024',
 'earthquakeBohol2013': 'TRECIS-CTIT-H-Test-025',
 'hurricaneFlorence2018': 'TRECIS-CTIT-H-Test-026',
 'shootingDallas2017': 'TRECIS-CTIT-H-Test-027',
 'fireYMM2016': 'TRECIS-CTIT-H-Test-028'}

In [17]:
tweet_id_to_topic_map = {}
tweet_id_to_count_map = {}

for file_path in glob.iglob("../data/2019/*2019*test*.json.gz"):
    print(file_path)
    counter = 1
    with gzip.open(file_path, "rb") as in_file:
        for line_ in in_file:
            line = line_.decode("utf8")
            tweet_entry = json.loads(line)
            
            tweet_id = np.int64(tweet_entry["allProperties"]["id"])
            tweet_topic = tweet_entry["topic"]
            
            if ( '1100409564803956744' == tweet_entry["allProperties"]["id"] ):
                print("Found")
                print(tweet_entry["allProperties"]["id"], tweet_id, tweet_entry["allProperties"]["id"] == tweet_id)
            
            tweet_id_to_topic_map[tweet_id] = topic_num_to_id_map[tweet_topic]
            tweet_id_to_count_map[tweet_id] = counter
            
            counter += 1

../data/2019/trecis2019-A-test.hurricaneFlorence2018.json.gz
../data/2019/trecis2019-A-test.fireYMM2016.json.gz
../data/2019/trecis2019-A-test.shootingDallas2017.json.gz
../data/2019/trecis2019-A-test.floodChoco2019.json.gz
../data/2019/trecis2019-A-test.earthquakeBohol2013.json.gz
../data/2019/trecis2019-A-test.earthquakeCalifornia2014.json.gz
../data/2019/trecis2019-A-test.fireAndover2019.json.gz
Found
1100409564803956744 1100409564803956744 False


In [18]:
len(tweet_id_to_count_map), priority_df.shape[0]

(9308, 9503)

In [19]:
category_map = {row["tweet_id"]: row["label"] for idx, row in category_df.iterrows()}
priority_map = {row["tweet_id"]: row["priority"] for idx, row in priority_df.iterrows()}

In [20]:
category_df["inc_id"] = category_df["tweet_id"].apply(tweet_id_to_topic_map.get)
category_df["count"] = category_df["tweet_id"].apply(tweet_id_to_count_map.get)
category_df["priority"] = category_df["tweet_id"].apply(priority_map.get)

In [21]:
priority_scorer = {
    1 : 0.75,
    0 : 0.25
}

with open("nyu-smapp_run_%s.csv" % run_title, "w") as out_file:
    for row in category_df.drop_duplicates(subset="tweet_id").itertuples():
        content = [
            row.inc_id, 
            "Q0", 
            row.tweet_id,
            row.count,
            priority_scorer[row.priority],
            str([row.label]).replace("'", '"'),
            "nyu-smapp_%s" % run_title
        ]
        out_file.write("\t".join([str(x) for x in content]) + "\n")